In [88]:
import numpy as np
import os
import timeit
import glob 
import pandas as pd
import time
import multiprocessing as mp 
import logging
import sys 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plt 

In [89]:
def get_junction_coordinates(df, coordinates_col, sep=':'):
    df['strand'] = None
    df['junction_coordinate'] = None

    for idx, row in df.iterrows():
        kmer_coordinates = [int(x) for x in row[coordinates_col].split(sep) if (x !='None') ]

        if kmer_coordinates[1] < kmer_coordinates[2]: # order strand +

            df.loc[idx, 'strand'] = '+'
            if len(kmer_coordinates) == 4:  # 2 exons
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:3]])
            elif len(kmer_coordinates) == 6:
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:5]])
        else: # order strand +
            df.loc[idx, 'strand'] = '-'
            if len(kmer_coordinates) == 4:  # 2 exons
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
                                                                                kmer_coordinates[0]]])
            elif len(kmer_coordinates) == 6:
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
                                                                                kmer_coordinates[0],
                                                                                kmer_coordinates[2],
                                                                                kmer_coordinates[5]
                                                                               ]])
    return df




### Get filtered data

In [90]:
# Inputs
run_type = 'ov'

if run_type == 'brca':
    target_samples = ['TCGA-C8-A12P-01A-11R-A115-07.all',
                      'TCGA-AO-A0JM-01A-21R-A056-07.all',
                      'TCGA-BH-A18V-01A-11R-A12D-07.all',
                      'TCGA-A2-A0D2-01A-21R-A034-07.all',
                      'TCGA-A2-A0SX-01A-12R-A084-07.all']
    sample_target = 'TCGA-AO-A0JM-01A-21R-A056-07'
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102'
    intermediate_output = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_order_r.tsv.gz')
elif run_type == 'ov':
    target_samples = ['TCGA-25-1319-01A-01R-1565-13.all',
                      'TCGA-25-1313-01A-01R-1565-13.all',
                      'TCGA-61-2008-01A-02R-1568-13.all',
                      'TCGA-24-1431-01A-01R-1566-13.all',
                      'TCGA-24-2298-01A-01R-1569-13.all']
    sample_target = 'TCGA-25-1319-01A-01R-1565-13'
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374'
    intermediate_output = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_order_r.tsv.gz')


# Outputs
filtering_id = 'filters_22March_order_wany_wAnnot'
output_dir = os.path.join(basedir, 'filtering_samples', filtering_id)

In [91]:
# Load generation matrix
df_load = pd.read_csv(intermediate_output, sep = '\t')

In [92]:
# Load filtered kmers
path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1.tsv.gz'
path_interest = os.path.join(output_dir, path_interest)
filt = pd.read_csv(path_interest, sep = '\t')
print(filt.shape)

(131575, 4)


In [93]:
# Merge generation matrix & filtered kmers
filt_meta = df_load.merge(filt, on = list(filt.columns), how = 'right')
print(filt_meta.shape)

(131962, 24)


In [94]:
# Define problematic kmer
filt_meta_pb = filt_meta

In [95]:
# Columns magic
core_cols = ['kmer', 'gtexCohortfilter >0.0', 'coord',
       'junctionAnnotated', 'readFrameAnnotated', 'isCrossJunction', 'batch',
       'cancerCohortfilter >0.0', sample_target.replace('-', '') + 'all', 'isAnnotated']

df_load.columns

Index(['kmer', 'gtexCohortfilter >0.0', 'gtexCohortfilter >=1.0',
       'gtexCohortfilter >=2.0', 'gtexCohortfilter >=3.0',
       'gtexCohortfilter >=5.0', 'gtexCohortfilter >=10.0', 'coord',
       'junctionAnnotated', 'readFrameAnnotated', 'isCrossJunction', 'batch',
       'cancerCohortfilter >0.0', 'cancerCohortfilter >=1.0',
       'cancerCohortfilter >=2.0', 'cancerCohortfilter >=3.0',
       'cancerCohortfilter >=5.0', 'cancerCohortfilter >=10.0',
       'TCGA25131901A01R156513all', 'TCGA25131301A01R156513all',
       'TCGA61200801A02R156813all', 'TCGA24143101A01R156613all',
       'TCGA24229801A01R156913all', 'isAnnotated'],
      dtype='object')

In [96]:
# Extract coordinates of the merged table
filt_meta_pb = get_junction_coordinates(filt_meta_pb, 'coord', sep=':')

display(filt_meta_pb[core_cols + ['junction_coordinate']].head())

,kmer,gtexCohortfilter >0.0,coord,junctionAnnotated,readFrameAnnotated,isCrossJunction,batch,cancerCohortfilter >0.0,TCGA25131901A01R156513all,isAnnotated,junction_coordinate
0,RMREATKGP,NaN,55350885:55350897:55352540:55352555:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
1,QVERMREAT,NaN,55350876:55350897:55352540:55352546:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
2,VERMREATK,NaN,55350879:55350897:55352540:55352549:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
3,MREATKGPV,NaN,55350888:55350897:55352540:55352558:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
4,VQVERMREA,NaN,55350873:55350897:55352540:55352543:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540


### Add the peptide metadata

In [97]:
# Define peptide file columns of interest 
cols_correct = ['peptide','id','readFrame','readFrameAnnotated','geneName','geneChr','geneStrand',
'mutationMode','hasStopCodon','isInJunctionList',
'isIsolated','variantComb','variantSegExpr','modifiedExonsCoord',
'originalExonsCoord',
'vertexIdx','kmerType', 'dummy1', 'dummy2', 'dummy3']

cols_pep_file = ['peptide', 'id', 'readFrame', 'geneName',
       'geneChr', 'geneStrand', 'mutationMode',
       'hasStopCodon', 'isInJunctionList', 'isIsolated',
       'variantSegExpr', 'modifiedExonsCoord', 'originalExonsCoord',
       'vertexIdx', 'kmerType']


cols_pep_file = {col:  'gtex_' + col for col in cols_pep_file}


In [98]:
len(filt_meta_pb['batch'].unique()) # Now 121 brca, # 2792 OV

2792

In [56]:
filt_meta_pb.head()

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=5.0,cancerCohortfilter >=10.0,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate
0,RMREATKGP,NaN,NaN,NaN,NaN,NaN,NaN,55350885:55350897:55352540:55352555:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
1,QVERMREAT,NaN,NaN,NaN,NaN,NaN,NaN,55350876:55350897:55352540:55352546:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
2,VERMREATK,NaN,NaN,NaN,NaN,NaN,NaN,55350879:55350897:55352540:55352549:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
3,MREATKGPV,NaN,NaN,NaN,NaN,NaN,NaN,55350888:55350897:55352540:55352558:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
4,VQVERMREA,NaN,NaN,NaN,NaN,NaN,NaN,55350873:55350897:55352540:55352543:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540


### Make False positive hypothesis

In [57]:
gtex_dir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref'
check_hypothesis2 = []
check_hypothesis1 = []
kmers_not_in_gtex = []
kmers_no_metadata = []

for batch_i, batch in enumerate(filt_meta_pb['batch'].unique()):
    print(f'\n Iteration {batch_i} batch {batch}')
    # CANCER SIDE GET BATCH Junctions
    batch_kmers_init = filt_meta_pb.loc[filt_meta_pb['batch'] == batch]
    print(f'Size cancer kmers-junctions {len(batch_kmers_init)}')
    
    # Read GTEX peptide file
    pep_path = os.path.join(gtex_dir, 'cohort_mutNone', f'tmp_out_ref_batch_{batch}', 'ref_sample_peptides_meta.gz')
#     if os.path.exists(pep_path):
    ##### COPY 
    not_in_background = False


    print(pep_path)
    try:
        meta_pep = pd.read_csv(pep_path, sep = '\t')
    except:
        print(f'CHECK COMPLETION OF {meta_pep}')
        kmers_no_metadata.append(batch_kmers_init[['kmer', 'junction_coordinate']])
        continue
    print(meta_pep.shape)

    meta_pep.columns = cols_correct # ISSUE WITH IMMUNOPEPPER



    # Only bi or tri-exons peptides
    jx_ids = [idx for idx, coord in enumerate(meta_pep['modifiedExonsCoord']) if 'nan' not in coord]
    meta_pep = meta_pep.iloc[jx_ids]

    # Quick Assess Cancer junction presence in GTEX
    coord_int = [i.split(':') for i in batch_kmers_init['junction_coordinate'].unique()]
    keep = set()
    for cd in coord_int:
        keep.update([idx for idx, coord in enumerate(meta_pep['modifiedExonsCoord']) \
                     if (cd[0] in coord) and (cd[1] in coord)])
    meta_pep = meta_pep.iloc[list(keep)]

    if keep: # Some target junctions are in "potentially" found in gtex 
        # Rename
        meta_pep = meta_pep.rename(cols_pep_file, axis = 1)

        # Extract coordinates peptides
        meta_pep = get_junction_coordinates(meta_pep, 'gtex_modifiedExonsCoord', sep=';')

        # Add Peptide info 
        meta_pep = meta_pep[list(cols_pep_file.values()) + ['junction_coordinate']].drop_duplicates()

        exist_jx_not_in_gtex = set(batch_kmers_init['junction_coordinate']).difference(set(meta_pep['junction_coordinate']))
        exist_jx_in_gtex = set(batch_kmers_init['junction_coordinate']).intersection(set(meta_pep['junction_coordinate']))

        if exist_jx_in_gtex: # Some target junctions are in gtex
            batch_kmers = batch_kmers_init.merge(meta_pep, on = 'junction_coordinate', how = 'inner')
            print(f'Size cancer kmers-junctions + all GTEX peptides {len(batch_kmers)}')

            # Calculate the number of aa fitting on each side of the junctions
            batch_kmers['gtex_aa_E1'] = batch_kmers['gtex_modifiedExonsCoord'].str.split(';').map(lambda x: abs(int(x[0]) - int(x[1]))/3 )
            batch_kmers['gtex_aa_E2'] = batch_kmers['gtex_modifiedExonsCoord'].str.split(';').map(lambda x: abs(int(x[2]) - int(x[3]))/3 )
            batch_kmers['cancer_aa_E1'] = batch_kmers['coord'].str.split(':').map(lambda x: abs(int(x[0]) - int(x[1]))/3 )
            batch_kmers['cancer_aa_E2'] = batch_kmers['coord'].str.split(':').map(lambda x: abs(int(x[2]) - int(x[3]))/3 )
            # What about 3 exons?

            # Validate hypothesis: 
            # H1: The second exon is not long enough to get the translation through in GTEX. BUT no exon was added on the right
            H_cols = ['gtexE2<cancE2', 'gtexE1<cancE1']
            for new_col in H_cols:
                batch_kmers[new_col] = True
            batch_kmers.loc[batch_kmers['gtex_aa_E2'] > batch_kmers['cancer_aa_E2'], 'gtexE2<cancE2'] = False
            batch_kmers.loc[batch_kmers['gtex_aa_E1'] > batch_kmers['cancer_aa_E1'], 'gtexE1<cancE1'] = False

            res = batch_kmers[['kmer', 'junction_coordinate' ] + H_cols].drop_duplicates()

            display( batch_kmers[['kmer', 'cancerCohortfilter >0.0',\
                                  sample_target.replace('-', '') + 'all',
                                  'readFrameAnnotated', \
                                  'junctionAnnotated', \
                                 ] + H_cols].drop_duplicates() )

            check_hypothesis2.append(res.loc[res['gtexE2<cancE2'] == True, ['kmer', 'junction_coordinate']])
            check_hypothesis1.append(res.loc[res['gtexE1<cancE1'] == True, ['kmer', 'junction_coordinate']])
            

        if exist_jx_not_in_gtex: # Some target junctions are NOT in gtex (Immunopepper Metadata)
            not_in_background = True
            diff = batch_kmers_init.set_index('junction_coordinate').loc[exist_jx_not_in_gtex].reset_index() #left anti join


    else: # No target junctions are in GTEX (Immunopepper Metadata) at all [CAREFUL - could be in graph]
        not_in_background = True
        diff = batch_kmers_init.copy()

    if not_in_background:
        #diff = diff[['kmer', 'junction_coordinate', 'cancerCohortfilter >0.0', ]].drop_duplicates()
        kmers_not_in_gtex.append(diff[['kmer', 'junction_coordinate']])
        print(f'{diff.shape[0]} Kmers - junctions not found in gtex. Recurrence is:')
        print(diff['cancerCohortfilter >0.0'].unique())
#             for rec in diff['cancerCohortfilter >0.0'].unique():
#                 if rec > 100: 
#                     display( diff[['kmer', 'junction_coordinate', 'coord', \
#                                    'cancerCohortfilter >0.0',\
#                                       sample_target.replace('-', '') + 'all',
#                                       'readFrameAnnotated', \
#                                       'junctionAnnotated' ]].drop_duplicates() )
        print(f'{diff.shape[0]} Kmers - junctions not found in gtex. Junction annotated is:')
        print(diff['junctionAnnotated'].unique())
        print(f'{diff.shape[0]} Kmers - junctions not found in gtex. RF annotated is:')
        print(diff['readFrameAnnotated'].unique())


    ##### END COPY 
#     else:
#         check_path = os.path.join(gtex_dir, 'cohort_mutNone', f'tmp_out_ref_batch_{batch}')
#         kmers_no_metadata.append(batch_kmers_init[['kmer', 'junction_coordinate']])
# #         for rec in diff['cancerCohortfilter >0.0'].unique():
# #             if rec > 1000: 
# #                 display(batch_kmers_init[['kmer', 'junction_coordinate']])
#         print(f'CHECK COMPLETION OF {check_path}')
#         continue
# #         for pep_path in glob.glob(pep_path + '/*'):





 Iteration 0 batch 16543
Size cancer kmers-junctions 7
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_16543/ref_sample_peptides_meta.gz
(15185, 20)
7 Kmers - junctions not found in gtex. Recurrence is:
[2]
7 Kmers - junctions not found in gtex. Junction annotated is:
[False]
7 Kmers - junctions not found in gtex. RF annotated is:
[ True]

 Iteration 1 batch 12597
Size cancer kmers-junctions 4
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_12597/ref_sample_peptides_meta.gz
(31021, 20)
4 Kmers - junctions not found in gtex. Recurrence is:
[4]
4 Kmers - junctions not found in gtex. Junction annotated is:
[False]
4 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 2 batch 41047
Size cancer kmers-junctions 6
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generati

,kmer,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated,gtexE2<cancE2,gtexE1<cancE1
0,SRIPCPSQL,33,5.875132,False,False,True,False
9,RIPCPSQLP,33,5.875132,False,False,True,False
18,GSPAPHSCR,33,5.875132,False,False,True,False
27,IPCPSQLPA,33,5.875132,False,False,True,False
36,SRIPCPSQN,143,5.875132,False,True,True,False
81,IPCPSQNTS,143,5.875132,False,True,True,False
126,PGSPAPHRI,143,5.875132,False,True,True,False
171,GSPAPHRIH,143,5.875132,False,True,True,False
216,RIPCPSQNT,143,5.875132,False,True,True,False
261,IPCPSQEKV,20,1.678609,False,False,True,False


87 Kmers - junctions not found in gtex. Recurrence is:
[12 89  4 16  8 26  5 24]
87 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
87 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 4 batch 416
Size cancer kmers-junctions 19
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_416/ref_sample_peptides_meta.gz
(33104, 20)
19 Kmers - junctions not found in gtex. Recurrence is:
[14  4 17]
19 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
19 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 5 batch 27568
Size cancer kmers-junctions 122
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_27568/ref_sample_peptides_meta.gz
(11126, 20)
122 Kmers - junctions not found in gtex. Recurrence is:
[42 12 20 17 28 44

(29614, 20)
442 Kmers - junctions not found in gtex. Recurrence is:
[ 16  26   1   9  78  53  13  37  10   2  52 165  12   8  50  75  48  89
  27   6 108   4  45  39  40 238 111  24  33  18  11  14 164  23]
442 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
442 Kmers - junctions not found in gtex. RF annotated is:
[ True False]

 Iteration 23 batch 26541
Size cancer kmers-junctions 13
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_26541/ref_sample_peptides_meta.gz
(43335, 20)
13 Kmers - junctions not found in gtex. Recurrence is:
[15 21]
13 Kmers - junctions not found in gtex. Junction annotated is:
[False]
13 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 24 batch 28631
Size cancer kmers-junctions 160
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out

(43683, 20)
547 Kmers - junctions not found in gtex. Recurrence is:
[206   3 341 114  62 218 225  38  84  14  79 200 310   6   7 166  10  27
  26  77  18 158 215 243  16   4  31 124 216 292  36  34 128 105  19 300
 148  51 319  53   8  23  61  37  44  67  54 165 250  22   2]
547 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
547 Kmers - junctions not found in gtex. RF annotated is:
[False  True]

 Iteration 31 batch 25398
Size cancer kmers-junctions 116
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_25398/ref_sample_peptides_meta.gz
(64019, 20)
116 Kmers - junctions not found in gtex. Recurrence is:
[22 47 26 34 17 44  1 50 18]
116 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
116 Kmers - junctions not found in gtex. RF annotated is:
[False  True]

 Iteration 32 batch 940
Size cancer kmers-junctions 6
/cluster/work/grlab/projects/proj

...

,kmer,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated,gtexE2<cancE2,gtexE1<cancE1
0,PKEKGDRGD,140,0.839305,False,False,True,False
6,KEKGDRGDA,140,0.839305,False,False,True,False


363 Kmers - junctions not found in gtex. Recurrence is:
[ 82  26   8  91 145  86  73  11  24  14  44 200  74  30  45  71  17  18
  23  50  15  27  58  21 115   7  43]
363 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
363 Kmers - junctions not found in gtex. RF annotated is:
[False  True]

 Iteration 2758 batch 34678
Size cancer kmers-junctions 6
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_34678/ref_sample_peptides_meta.gz
(34837, 20)
6 Kmers - junctions not found in gtex. Recurrence is:
[6]
6 Kmers - junctions not found in gtex. Junction annotated is:
[False]
6 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 2759 batch 11316
Size cancer kmers-junctions 54
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_11316/ref_sample_peptides_meta.gz


,kmer,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated,gtexE2<cancE2,gtexE1<cancE1
0,PADCKYNAQ,133,0.839305,False,False,False,True
29,SPADCKYNA,133,0.839305,False,False,False,True
58,LSPADCKYN,133,0.839305,False,False,False,True


63 Kmers - junctions not found in gtex. Recurrence is:
[ 56 140  78 204 123  68]
63 Kmers - junctions not found in gtex. Junction annotated is:
[False]
63 Kmers - junctions not found in gtex. RF annotated is:
[False  True]

 Iteration 2764 batch 57539
Size cancer kmers-junctions 7
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_57539/ref_sample_peptides_meta.gz
(1341, 20)
7 Kmers - junctions not found in gtex. Recurrence is:
[2]
7 Kmers - junctions not found in gtex. Junction annotated is:
[False]
7 Kmers - junctions not found in gtex. RF annotated is:
[ True]

 Iteration 2765 batch 21173
Size cancer kmers-junctions 22
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_21173/ref_sample_peptides_meta.gz
(36921, 20)
22 Kmers - junctions not found in gtex. Recurrence is:
[19 12]
22 Kmers - junctions

(438826, 20)
6 Kmers - junctions not found in gtex. Recurrence is:
[19]
6 Kmers - junctions not found in gtex. Junction annotated is:
[False]
6 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 2782 batch 47550
Size cancer kmers-junctions 5
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_47550/ref_sample_peptides_meta.gz
(9646, 20)
5 Kmers - junctions not found in gtex. Recurrence is:
[4]
5 Kmers - junctions not found in gtex. Junction annotated is:
[False]
5 Kmers - junctions not found in gtex. RF annotated is:
[ True]

 Iteration 2783 batch 3402
Size cancer kmers-junctions 467
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_3402/ref_sample_peptides_meta.gz
(65555, 20)
467 Kmers - junctions not found in gtex. Recurrence is:
[150  98  24  86 148  29  22  16 233   9  23 

In [120]:
print(filt_meta_pb.shape)
print(filt_meta_pb[['kmer', 'junction_coordinate']].drop_duplicates().shape)

(131962, 26)
(119786, 2)


In [121]:
df_kmers_no_metadata = pd.concat(kmers_no_metadata, axis = 0 )
print(f'{df_kmers_no_metadata.shape[0]} kmers without metadata')
print(f'{df_kmers_no_metadata.drop_duplicates().shape[0]} UNIQUE kmers without metadata')

5618 kmers without metadata
4748 UNIQUE kmers without metadata


In [122]:
df_kmers_not_in_gtex = pd.concat(kmers_not_in_gtex, axis = 0 )
print(f'{df_kmers_not_in_gtex.shape[0]} kmers completely absent in GTEX')
print(f'{df_kmers_not_in_gtex.drop_duplicates().shape[0]} UNIQUE kmers completely absent in GTEX')

125490 kmers completely absent in GTEX
114240 UNIQUE kmers completely absent in GTEX


In [123]:
df_check_hypothesis1 = pd.concat(check_hypothesis1, axis = 0 )
print(f'{df_check_hypothesis1.shape[0]} kmers coming from novel exons')
print(f'{df_check_hypothesis1.drop_duplicates().shape[0]} UNIQUE kmers coming from novel exons')

169 kmers coming from novel exons
156 UNIQUE kmers coming from novel exons


In [124]:
df_check_hypothesis2 = pd.concat(check_hypothesis2, axis = 0 )
print(f'{df_check_hypothesis2.shape[0]} kmers coming from novel exons')
print(f'{df_check_hypothesis2.drop_duplicates().shape[0]} UNIQUE kmers coming from novel exons')

663 kmers coming from novel exons
641 UNIQUE kmers coming from novel exons


In [125]:
merge_base = filt_meta_pb.copy()

# Create new labels based on the false positive check above 
# The sets no_metadata, not_GTEX, check_hypothesis are disjoint
print(set(df_kmers_no_metadata['kmer']).intersection( set(df_kmers_not_in_gtex['kmer'])))
print(set(df_kmers_no_metadata['kmer']).intersection( set(df_check_hypothesis1['kmer'])))
print(set(df_kmers_no_metadata['kmer']).intersection( set(df_check_hypothesis2['kmer'])))
print(len(set(df_kmers_not_in_gtex['kmer']).intersection( set(df_check_hypothesis1['kmer']))))
print(len(set(df_kmers_not_in_gtex['kmer']).intersection( set(df_check_hypothesis2['kmer']))))

# Drop duplicates
df_kmers_no_metadata = df_kmers_no_metadata.drop_duplicates() 
df_kmers_not_in_gtex = df_kmers_not_in_gtex.drop_duplicates()
df_check_hypothesis1 = df_check_hypothesis1.drop_duplicates()
df_check_hypothesis2 = df_check_hypothesis2.drop_duplicates()



# Augment df with information about GTEX junction presence
df_no_info = df_kmers_no_metadata.copy()
df_no_info['coordinate_in_GTEX'] = 'not_available' # No info
df_kmers_not_in_gtex['coordinate_in_GTEX'] = False
df_coord_in_gtex = pd.concat([df_kmers_not_in_gtex, df_no_info], axis = 0)

# Augment df with information about Novel exon hypothesis
df_no_info = df_kmers_no_metadata.copy()
df_no_info['gtexExon1<cancExon1'] = 'not_available'
df_check_hypothesis1['gtexExon1<cancExon1'] = True
df_exon_length1 = pd.concat([df_check_hypothesis1, df_no_info], axis = 0)

df_no_info = df_kmers_no_metadata.copy()
df_no_info['gtexExon2<cancExon2'] = 'not_available'
df_check_hypothesis2['gtexExon2<cancExon2'] = True
df_exon_length2 = pd.concat([df_check_hypothesis2, df_no_info], axis = 0)

# Augment with information about metadata availability 
df_kmers_no_metadata['info_not_available'] = True

# Merge
merge_base = merge_base.merge(df_coord_in_gtex, on = ['kmer', 'junction_coordinate'], how = 'left')
merge_base['coordinate_in_GTEX'] = merge_base['coordinate_in_GTEX'].fillna(True)

merge_base = merge_base.merge(df_exon_length1, on = ['kmer', 'junction_coordinate'], how = 'left')
merge_base['gtexExon1<cancExon1'] = merge_base['gtexExon1<cancExon1'].fillna(False)

merge_base = merge_base.merge(df_exon_length2, on = ['kmer', 'junction_coordinate'], how = 'left')
merge_base['gtexExon2<cancExon2'] = merge_base['gtexExon2<cancExon2'].fillna(False)

merge_base = merge_base.merge(df_kmers_no_metadata, on = ['kmer', 'junction_coordinate'], how = 'left')
merge_base['info_not_available'] = merge_base['info_not_available'].fillna(False)

display(merge_base.groupby('gtexExon2<cancExon2').count())
display(merge_base.groupby('gtexExon1<cancExon1').count())
display(merge_base.groupby('coordinate_in_GTEX').count())
display(merge_base.groupby('info_not_available').count())

set()
set()
set()
1
53


,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate,coordinate_in_GTEX,gtexExon1<cancExon1,info_not_available
gtexExon2<cancExon2,,,,,,,,,,,,,,,,,,,,,
False,125657,242,242,242,242,242,242,125657,125657,125657,...,125657,125657,125657,125657,6,125657,125657,125657,125657,125657
True,687,37,37,37,37,37,37,687,687,687,...,687,687,687,687,8,687,687,687,687,687
not_available,5618,404,404,404,404,404,404,5618,5618,5618,...,5618,5618,5618,5618,8,5618,5618,5618,5618,5618


,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate,coordinate_in_GTEX,gtexExon2<cancExon2,info_not_available
gtexExon1<cancExon1,,,,,,,,,,,,,,,,,,,,,
False,126180,279,279,279,279,279,279,126180,126180,126180,...,126180,126180,126180,126180,14,126180,126180,126180,126180,126180
True,164,0,0,0,0,0,0,164,164,164,...,164,164,164,164,0,164,164,164,164,164
not_available,5618,404,404,404,404,404,404,5618,5618,5618,...,5618,5618,5618,5618,8,5618,5618,5618,5618,5618


,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate,gtexExon1<cancExon1,gtexExon2<cancExon2,info_not_available
coordinate_in_GTEX,,,,,,,,,,,,,,,,,,,,,
False,125496,183,183,183,183,183,183,125496,125496,125496,...,125496,125496,125496,125496,6,125496,125496,125496,125496,125496
True,848,96,96,96,96,96,96,848,848,848,...,848,848,848,848,8,848,848,848,848,848
not_available,5618,404,404,404,404,404,404,5618,5618,5618,...,5618,5618,5618,5618,8,5618,5618,5618,5618,5618


,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate,coordinate_in_GTEX,gtexExon1<cancExon1,gtexExon2<cancExon2
info_not_available,,,,,,,,,,,,,,,,,,,,,
False,126344,279,279,279,279,279,279,126344,126344,126344,...,126344,126344,126344,126344,14,126344,126344,126344,126344,126344
True,5618,404,404,404,404,404,404,5618,5618,5618,...,5618,5618,5618,5618,8,5618,5618,5618,5618,5618


In [128]:
path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1.tsv.gz'
path_interest_with_advanced_meta = os.path.join(output_dir, path_interest.replace('.tsv.gz', 'metadata2.tsv.gz'))
print(f'Saving kmers with advanced metadata to {path_interest_with_advanced_meta}')
merge_base.to_csv(path_interest_with_advanced_meta, sep = '\t', compression = 'gzip', index = None)

Saving kmers with advanced metadata to /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_22March_order_wany_wAnnot/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1metadata2.tsv.gz


## Exploratory

### Remove GTEX annotation 

In [19]:
# path_annot = glob.glob('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_6920432_ANNOT_conf2_RFall_ref/cohort_mutNone/*/ref_annot_kmer.gz')

# print(len(path_annot))

In [20]:
# for idx, annot in enumerate(path_annot):
#     kmers_filter_pipeline = len(kmer_post_filter)
#     annot = pd.read_csv(annot, sep = '\t')
#     annot = set(annot['kmer'])
#     kmer_post_filter = kmer_post_filter.difference(annot)
#     if kmers_filter_pipeline != len(kmer_post_filter):
#         print(path_annot[idx], len(kmer_post_filter))